In [ ]:
import pandas as pd

# Charger tes logs réseau
data = pd.read_csv('logs_reseau.csv', parse_dates=['Date'])

# Convertir 'Date' en un format datetime si ce n'est pas déjà le cas
data['Date'] = pd.to_datetime(data['Date'])

# Exemple de structure des données après le chargement
data.head()


In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Extraire les caractéristiques à analyser pour le clustering
# Par exemple : Adresse IP source, port de destination, protocole, action
features = data[['Adresse IP Source', 'Port de destination', 'Protocole', 'Action']]

# Il faut encoder les variables non numériques (par exemple, Protocole et Action)
features['Protocole'] = features['Protocole'].astype('category').cat.codes
features['Action'] = features['Action'].astype('category').cat.codes

# Standardiser les données
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Appliquer K-Means (choisir k en fonction du cas d'utilisation)
kmeans = KMeans(n_clusters=3, random_state=42)
data['Cluster'] = kmeans.fit_predict(scaled_features)

# Vérification des clusters créés
data[['Adresse IP Source', 'Port de destination', 'Protocole', 'Action', 'Cluster']].head()


In [ ]:
# Calcul des connexions par adresse IP source et par période (par exemple, par heure)
data['Hour'] = data['Date'].dt.hour
traffic_by_ip = data.groupby(['Adresse IP Source', 'Hour']).size().reset_index(name='Connection Count')

# Détecter des anomalies en observant un grand nombre de connexions par heure
threshold = traffic_by_ip['Connection Count'].quantile(0.95)  # Limite à 95% des connexions les plus élevées
anomalies = traffic_by_ip[traffic_by_ip['Connection Count'] > threshold]

# Affichage des anomalies détectées
anomalies.head()


In [ ]:
import matplotlib.pyplot as plt

# Calculer le trafic total par période de temps
traffic_by_time = data.groupby(data['Date'].dt.hour).size()

# Tracer les pics de trafic
plt.figure(figsize=(10,6))
plt.plot(traffic_by_time.index, traffic_by_time.values, label='Nombre de Connexions par Heure')
plt.xlabel('Heure')
plt.ylabel('Nombre de Connexions')
plt.title('Pic de Trafic Réseau par Heure')
plt.grid(True)
plt.show()

# Identifier les pics de trafic
pic_threshold = traffic_by_time.quantile(0.95)  # Limite à 95% du pic
peak_hours = traffic_by_time[traffic_by_time > pic_threshold]
print("Heures de pics de trafic : ", peak_hours)


In [ ]:
import seaborn as sns

# Carte thermique de la répartition des clusters par heure et IP source
plt.figure(figsize=(10,6))
heatmap_data = data.pivot_table(index='Adresse IP Source', columns='Hour', values='Cluster', aggfunc='first')
sns.heatmap(heatmap_data, cmap='coolwarm', annot=True)
plt.title('Carte Thermique des Clusters d\'Activités')
plt.xlabel('Heure')
plt.ylabel('Adresse IP Source')
plt.show()

In [ ]:
# Courbes de détection de pics par heure
plt.figure(figsize=(10,6))
sns.lineplot(x=traffic_by_time.index, y=traffic_by_time.values, label="Nombre de Connexions par Heure")
sns.lineplot(x=peak_hours.index, y=peak_hours.values, label="Pics de Trafic", color='red', linestyle='--')
plt.title('Courbes de Trafic et Détection des Pics')
plt.xlabel('Heure')
plt.ylabel('Nombre de Connexions')
plt.legend()
plt.show()